<a href="https://colab.research.google.com/github/Ibtisam-a/Project-Implementation/blob/master/3a_Similarity_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.sparse import diags

#Our parameters
#number of internal nodes , unknowns, in each direction

N = 190  # our values: 190, 380, 760, 1520, 3040, 6080 and 12160

#number of internal nodes, unknowns, in each direction
n = N-2 
#step size 
h = 1/(n+1) 

#Matrix with zeros
U = np.zeros([n,n])

# x and y arrays between 0 and 1, spaced internal points
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Internal mesh without boundaries
X, Y = np.meshgrid(x, y, indexing='ij')

#our function 
F = np.sin(2*np.pi*X)*np.sin(2*np.pi*Y)  

#Tridiagonal matrix A
diagonals = [[1.5],[-0.5],[-0.5]]
A = np.multiply(1, diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())

#Compute the exact solution to be comapred on different time steps
##0.0000001, 0.000000025, 0.00000000625, 0.0000000015625, 0.00000000039062, 0.00000000009766
Exact_sol = np.exp(-8*(np.pi*np.pi)*(0.0000001))*np.sin(2*np.pi*X)*np.sin(2*np.pi*Y)




In [2]:
import numpy as np
import time
from scipy.sparse.linalg import spsolve





def similarity_trans(A, F):
    n = len(A)
    start_time = time.time()
    eigvals, eigvecs = np.linalg.eig(A) #Get eigenvalues and eigenvectors of T
    eig_time = time.time() - start_time
    diag = diags(eigvals, 0, shape=(n, n)).toarray() #Set eigenvalues as diagonal matrix
    P = eigvecs #Set basis matrix P, with columns as the eigenvectors
    P_inv = np.linalg.inv(P)
    inv_time = time.time() - eig_time - start_time
    F_hat = np.matmul(np.matmul(P_inv,F),P) #F_hat = P^-1 * F * P
    U_hat = np.empty([n,n]) #u_hat(i,j) = f(i,j) / eigval(i) + eigval(j)
    for i in range(0,n):
        for j in range(0,n):
            if (eigvals[i]!=0 or eigvals[j]!=0):
                U_hat[i][j] = F_hat[i][j] / (eigvals[i]+eigvals[j])
            else:
                U_hat[i][j] = 0
    U = np.matmul(np.matmul(P,U_hat),P_inv) #U = P * U^hat * P^-1
    end_time = time.time()
    solve_time = end_time - inv_time - eig_time - start_time
    total_time = end_time - start_time
    all_timess = [eig_time, inv_time, solve_time, total_time]
    return U, all_timess

    

In [ ]:
import numpy as np

def compute_error(A, F):   
    n = len(U)
    # The maximum difference between both solutions
    err_max_difference = 0  
    # The average difference between both solutions 
    err_ave_difference = 0

#For loop to compute the error
    for i in range(0,n):
        for j in range(0,n):
            err_ave_difference += np.absolute(Exact_sol[i][j] - U[i][j])**2
            if np.absolute(Exact_sol[i][j] - U[i][j]) > err_max_difference:
                err_max_difference = np.absolute(Exact_sol[i][j] - U[i][j])       
    err_ave_difference = (err_ave_difference * h**2)**0.5
    return err_max_difference, err_ave_difference

#Solving our system.
U, all_timess = similarity_trans(A, F)   
err_max_difference, err_ave_difference = compute_error(U, Exact_sol)

print('Time taken:', all_timess)
print('Maximum diffence:', err_max_difference, '\n', 'Average difference:', err_ave_difference)